In [1]:
import numpy as np
from numpy import zeros, eye, array, diag, exp, ones
from numpy.linalg import solve, norm
from numpy.random import choice, default_rng, randint, randn
from scipy.stats import multivariate_normal as MVN
from scipy.special import logsumexp
from scipy.stats import uniform as udist
import math
import time
from copy import deepcopy

import matplotlib.pyplot as plt
from matplotlib import rc
from ipywidgets.widgets import IntSlider
from ipywidgets.widgets import interact
from warnings import resetwarnings

from Manifolds.GKManifoldNew import GKManifold, data_generator, find_point_on_manifold, find_point_on_manifold_from_θ
from markov_snippets import MSAdaptive, SMCAdaptive
from tangential_hug_functions import TangentialHugSampler
from RWM import RWM

In [2]:
seed = randint(low=1000, high=9999)
print("Using random seed: ", seed)

Using random seed:  2962


In [3]:
m = 50     # number of latent variables
d = m + 4  # dimensionality of x-component

In [6]:
def generate_settings(
    d=10, 
    N=5000,
    B=20,
    δ=0.01,
    δmin=1e-3,
    δmax=100.0,
    εmin=1e-16,
    min_pm=1e-4,
    maxiter=500,
    verbose=True,
    εs_fixed=None,
    adaptiveε=True,
    adaptiveδ=False,
    z0_manual=None,
    pm_target=0.23,
    pm_switch=0.1,
    prior_seed=seed,
    low_memory=True,
    integrator='rwm',
    εprop_switch=0.01,
    ε0_manual=None,
    quantile_value=0.9,
    initialization='manual',
    switch_strategy='pm',
    kernel='uniform',
    prior_scale=7,
    thinning=10,
    burnin=100,
    use_autograd=False,
    stopping_criterion={'maxiter', 'εmin', 'pm'}):
    # here d represents m+4
    # Generate the manifold
    θ0        = array([3.0, 1.0, 2.0, 0.5])       # True parameter value on U(0, 10) scale.
    #d         = 4 + m                            # Dimensionality of ξ=(θ, z)
    ystar     = data_generator(θ0, m, seed=1234)  # Observed data
    q         = MVN(zeros(d), eye(d))             # Proposal distribution for THUG
    resetwarnings()
    manifold  = GKManifold(ystar, kernel_type=kernel, use_autograd=use_autograd)
    # grab arguments
    arguments = locals()
    return arguments

# Manual Initialization

In [11]:
# Generate Manifold
θ0        = array([3.0, 1.0, 2.0, 0.5])       # True parameter value on U(0, 10) scale.
ystar     = data_generator(θ0, m, seed=1234)  # Observed data
resetwarnings()
manifold  = GKManifold(ystar, kernel_type='uniform', use_autograd=False)

# Generate target distribution
ξ0 = manifold.sample()               # Initial point on manifold  
ε0 = 10.0                            # Starting epsilon
logηε0 = manifold.generate_logηε(ε0)  # Target filamentary distribution

# Sample from it using RWM
s_rwm, a_rwm = RWM(x0=ξ0, s=0.001, N=5000, logpi=logηε0)

# Add velocities to particles
v0_manual = randn(*s_rwm.shape)
z0_manual = np.hstack((s_rwm, v0_manual))

In [18]:
SETTINGS_MANUAL_RWM_0001 = generate_settings(
    δ=0.001,
    d=d, 
    integrator='rwm', 
    initialization='manual', 
    adaptiveδ=False,
    adaptiveε=True,
    z0_manual=z0_manual, 
    ε0_manual=ε0,
    stopping_criterion={'maxiter', 'εmin', 'pm'},
    min_pm=0.0
)

SETTINGS_MANUAL_RWM_001 = generate_settings(
    δ=0.01,
    d=d, 
    integrator='rwm', 
    initialization='manual', 
    adaptiveδ=False,
    adaptiveε=True,
    z0_manual=z0_manual, 
    ε0_manual=ε0,
    stopping_criterion={'maxiter', 'εmin', 'pm'},
    min_pm=0.0
)

SETTINGS_MANUAL_RWM_ADAPTIVE = generate_settings(
    δ=0.01,
    d=d, 
    integrator='rwm', 
    initialization='manual', 
    adaptiveδ=True,
    adaptiveε=True,
    z0_manual=z0_manual, 
    ε0_manual=ε0,
    stopping_criterion={'maxiter', 'εmin', 'pm'},
    min_pm=0.0
)

SETTINGS_MANUAL_RT_0001 = generate_settings(
    δ=0.001,
    d=d, 
    integrator='rwm_then_thug', 
    initialization='manual', 
    adaptiveδ=False,
    adaptiveε=True,
    z0_manual=z0_manual, 
    ε0_manual=ε0,
    stopping_criterion={'maxiter', 'εmin', 'pm'},
    pm_switch=0.1,
    min_pm=0.0
)

# MS-RWM

In [14]:
MS_RWM_MAN_0001 = MSAdaptive(SETTINGS_MANUAL_RWM_0001)
_ = MS_RWM_MAN_0001.sample()

Integrator: RWM.
Stopping criterion:  maxiter, εmin, pm.
Iteration:  1
	Trajectories constructed.
	Epsilon: 9.9378205109852100
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9450000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  2
	Trajectories constructed.
	Epsilon: 9.9013082689762584
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9426000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  3
	Trajectories constructed.
	Epsilon: 9.8606516861393594
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9480000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  4
	Trajectories constructed.
	Epsilon: 9.8154994992595146
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9476000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  5
	Trajectories constructed

	Epsilon: 8.3949178668251303
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9498000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  38
	Trajectories constructed.
	Epsilon: 8.3614956155452020
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9482000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  39
	Trajectories constructed.
	Epsilon: 8.3293955303134162
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9450000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  40
	Trajectories constructed.
	Epsilon: 8.2956233389767604
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9480000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  41
	Trajectories constructed.
	Epsilon: 8.2640487225924950
	Weights computed and normalized.
	Particles Resampled.
	Veloci

	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9472000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  74
	Trajectories constructed.
	Epsilon: 7.3080147896564052
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9442000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  75
	Trajectories constructed.
	Epsilon: 7.2756357055831398
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9492000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  76
	Trajectories constructed.
	Epsilon: 7.2431847325133809
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9496000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  77
	Trajectories constructed.
	Epsilon: 7.2131577856674571
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 

	Epsilon: 6.3405526275380506
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9422000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  111
	Trajectories constructed.
	Epsilon: 6.3153714068714502
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9498000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  112
	Trajectories constructed.
	Epsilon: 6.2890820257883684
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9436000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  113
	Trajectories constructed.
	Epsilon: 6.2647585479103656
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9448000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  114
	Trajectories constructed.
	Epsilon: 6.2403894795235031
	Weights computed and normalized.
	Particles Resampled.
	Ve

	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9470000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  147
	Trajectories constructed.
	Epsilon: 5.4486369094853115
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9450000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  148
	Trajectories constructed.
	Epsilon: 5.4277581782561946
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9434000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  149
	Trajectories constructed.
	Epsilon: 5.4056669028509265
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9436000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  150
	Trajectories constructed.
	Epsilon: 5.3833184300158017
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Mov

...

	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9134000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  327
	Trajectories constructed.
	Epsilon: 2.5468394170664821
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9262000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  328
	Trajectories constructed.
	Epsilon: 2.5363139763887128
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9246000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  329
	Trajectories constructed.
	Epsilon: 2.5250545057235629
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9246000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  330
	Trajectories constructed.
	Epsilon: 2.5150654788952376
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Mov

	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9140000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  363
	Trajectories constructed.
	Epsilon: 2.2004330127317324
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9147999999999999
	Step-size kept fixed at: 0.0010000000000000
Iteration:  364
	Trajectories constructed.
	Epsilon: 2.1915001550664468
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9092000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  365
	Trajectories constructed.
	Epsilon: 2.1830453629203306
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9038000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  366
	Trajectories constructed.
	Epsilon: 2.1745374640031598
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Mov

	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9028000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  399
	Trajectories constructed.
	Epsilon: 1.9059884666041764
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9002000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  400
	Trajectories constructed.
	Epsilon: 1.8988577400328384
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9072000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  401
	Trajectories constructed.
	Epsilon: 1.8915820172085427
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.9004000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  402
	Trajectories constructed.
	Epsilon: 1.8850535023173709
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Mov

	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8895999999999999
	Step-size kept fixed at: 0.0010000000000000
Iteration:  435
	Trajectories constructed.
	Epsilon: 1.6729721793827808
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8966000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  436
	Trajectories constructed.
	Epsilon: 1.6661952764279493
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8774000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  437
	Trajectories constructed.
	Epsilon: 1.6597602816484545
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8934000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  438
	Trajectories constructed.
	Epsilon: 1.6537194611262944
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Mov

	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8692000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  471
	Trajectories constructed.
	Epsilon: 1.4725098382753721
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8738000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  472
	Trajectories constructed.
	Epsilon: 1.4670731725233954
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8722000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  473
	Trajectories constructed.
	Epsilon: 1.4627740059773180
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8652000000000000
	Step-size kept fixed at: 0.0010000000000000
Iteration:  474
	Trajectories constructed.
	Epsilon: 1.4580417880940102
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Mov

In [17]:
MS_RWM_MAN_001 = MSAdaptive(SETTINGS_MANUAL_RWM_001)
_ = MS_RWM_MAN_001.sample()

Integrator: RWM.
Stopping criterion:  maxiter, εmin, pm.
Iteration:  1
	Trajectories constructed.
	Epsilon: 9.9378205109852100
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8512000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  2
	Trajectories constructed.
	Epsilon: 9.8873337484744681
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8628000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  3
	Trajectories constructed.
	Epsilon: 9.8317639072518759
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8802000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  4
	Trajectories constructed.
	Epsilon: 9.7672800599032144
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8902000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  5
	Trajectories constructed

	Epsilon: 7.9917918155211414
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8618000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  38
	Trajectories constructed.
	Epsilon: 7.9577063173620912
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8590000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  39
	Trajectories constructed.
	Epsilon: 7.9265331708562643
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8606000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  40
	Trajectories constructed.
	Epsilon: 7.8970407857827496
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8632000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  41
	Trajectories constructed.
	Epsilon: 7.8665410196175714
	Weights computed and normalized.
	Particles Resampled.
	Veloci

	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8512000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  74
	Trajectories constructed.
	Epsilon: 6.2263056172564895
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8506000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  75
	Trajectories constructed.
	Epsilon: 6.1775781665180585
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8504000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  76
	Trajectories constructed.
	Epsilon: 6.1309519607824141
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8440000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  77
	Trajectories constructed.
	Epsilon: 6.0827696695240476
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 

	Epsilon: 4.7546012229330623
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8018000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  111
	Trajectories constructed.
	Epsilon: 4.7197655604071667
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.7926000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  112
	Trajectories constructed.
	Epsilon: 4.6874936108925240
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8018000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  113
	Trajectories constructed.
	Epsilon: 4.6534599701124586
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8128000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  114
	Trajectories constructed.
	Epsilon: 4.6235560005499208
	Weights computed and normalized.
	Particles Resampled.
	Ve

	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.7382000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  147
	Trajectories constructed.
	Epsilon: 3.7617398601571899
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.7388000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  148
	Trajectories constructed.
	Epsilon: 3.7368270438556785
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.7328000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  149
	Trajectories constructed.
	Epsilon: 3.7155234596408606
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.7360000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  150
	Trajectories constructed.
	Epsilon: 3.6954271284345563
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Mov

...

	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.3190000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  327
	Trajectories constructed.
	Epsilon: 1.3663794718099542
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2902000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  328
	Trajectories constructed.
	Epsilon: 1.3628489856061810
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.3220000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  329
	Trajectories constructed.
	Epsilon: 1.3593020346395661
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.3042000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  330
	Trajectories constructed.
	Epsilon: 1.3557645053366154
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Mov

	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2944000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  363
	Trajectories constructed.
	Epsilon: 1.2449657904616058
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2764000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  364
	Trajectories constructed.
	Epsilon: 1.2414724405313955
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2744000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  365
	Trajectories constructed.
	Epsilon: 1.2381788890180807
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2870000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  366
	Trajectories constructed.
	Epsilon: 1.2343877620430201
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Mov

	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.1618000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  399
	Trajectories constructed.
	Epsilon: 1.1512641843386016
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.1702000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  400
	Trajectories constructed.
	Epsilon: 1.1492342768950203
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.1654000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  401
	Trajectories constructed.
	Epsilon: 1.1474848630694232
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.1630000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  402
	Trajectories constructed.
	Epsilon: 1.1452202475534983
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Mov

	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.1174000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  435
	Trajectories constructed.
	Epsilon: 1.0913280621856840
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.1152000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  436
	Trajectories constructed.
	Epsilon: 1.0898260352348386
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.1242000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  437
	Trajectories constructed.
	Epsilon: 1.0884310864199191
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.1140000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  438
	Trajectories constructed.
	Epsilon: 1.0869567713064439
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Mov

	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.0662000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  471
	Trajectories constructed.
	Epsilon: 1.0495561630700874
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.0572000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  472
	Trajectories constructed.
	Epsilon: 1.0485435863684056
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.0632000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  473
	Trajectories constructed.
	Epsilon: 1.0476556456173827
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.0698000000000000
	Step-size kept fixed at: 0.0100000000000000
Iteration:  474
	Trajectories constructed.
	Epsilon: 1.0465758256367226
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Mov

In [ ]:
MS_RWM_MAN_ADAPT = MSAdaptive(SETTINGS_MANUAL_RWM_ADAPTIVE)
_ = MS_RWM_MAN_ADAPT.sample()

Integrator: RWM.
Stopping criterion:  maxiter, εmin, pm.
Iteration:  1
	Trajectories constructed.
	Epsilon: 9.9378205109852100
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8594000000000001
	Step-size adapted to: 0.0136984829481921
Iteration:  2
	Trajectories constructed.
	Epsilon: 9.8929085149930245
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8400000000000000
	Step-size adapted to: 0.0185837044707719
Iteration:  3
	Trajectories constructed.
	Epsilon: 9.8329039638311020
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.8076000000000000
	Step-size adapted to: 0.0248059884305068
Iteration:  4
	Trajectories constructed.
	Epsilon: 9.7641910504471010
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.7632000000000000
	Step-size adapted to: 0.0323846677432360
Iteration:  5
	Trajectories constructed.
	Epsilon: 

	Epsilon: 8.1446841390070635
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2732000000000000
	Step-size adapted to: 0.1235795518207913
Iteration:  39
	Trajectories constructed.
	Epsilon: 8.0974437866153899
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2894000000000000
	Step-size adapted to: 0.1273049122747016
Iteration:  40
	Trajectories constructed.
	Epsilon: 8.0359481049499095
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2750000000000000
	Step-size adapted to: 0.1302017399028142
Iteration:  41
	Trajectories constructed.
	Epsilon: 7.9756603720028920
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2670000000000000
	Step-size adapted to: 0.1326328908996804
Iteration:  42
	Trajectories constructed.
	Epsilon: 7.9190601156918916
	Weights computed and normalized.
	Particles Resampled.
	Velocities refresh

	Epsilon: 6.1069553952647144
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2022000000000000
	Step-size adapted to: 0.1081895837175497
Iteration:  76
	Trajectories constructed.
	Epsilon: 6.0588259871445596
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.1970000000000000
	Step-size adapted to: 0.1064191022261309
Iteration:  77
	Trajectories constructed.
	Epsilon: 6.0157288494232253
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2058000000000000
	Step-size adapted to: 0.1051391901730713
Iteration:  78
	Trajectories constructed.
	Epsilon: 5.9741484067816160
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2176000000000000
	Step-size adapted to: 0.1044893437994297
Iteration:  79
	Trajectories constructed.
	Epsilon: 5.9302189763184803
	Weights computed and normalized.
	Particles Resampled.
	Velocities refresh

	Epsilon: 4.8303508835638880
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2060000000000000
	Step-size adapted to: 0.0746107868793069
Iteration:  113
	Trajectories constructed.
	Epsilon: 4.8049504134069956
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2320000000000000
	Step-size adapted to: 0.0746854349840178
Iteration:  114
	Trajectories constructed.
	Epsilon: 4.7804067963343888
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2200000000000000
	Step-size adapted to: 0.0743129398230315
Iteration:  115
	Trajectories constructed.
	Epsilon: 4.7533635196654433
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2034000000000000
	Step-size adapted to: 0.0733311212894144
Iteration:  116
	Trajectories constructed.
	Epsilon: 4.7295029157785144
	Weights computed and normalized.
	Particles Resampled.
	Velocities ref

	Epsilon: 3.8955715102314428
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.1968000000000000
	Step-size adapted to: 0.0592452194868594
Iteration:  150
	Trajectories constructed.
	Epsilon: 3.8759608771677541
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.1992000000000000
	Step-size adapted to: 0.0583398324801082
Iteration:  151
	Trajectories constructed.
	Epsilon: 3.8570468656576367
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.1908000000000000
	Step-size adapted to: 0.0572075048238797
Iteration:  152
	Trajectories constructed.
	Epsilon: 3.8372593425699462
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2096000000000000
	Step-size adapted to: 0.0566269541166489
Iteration:  153
	Trajectories constructed.
	Epsilon: 3.8155132832103393
	Weights computed and normalized.
	Particles Resampled.
	Velocities ref

	Trajectories constructed.
	Epsilon: 3.2640610814567879
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2396000000000000
	Step-size adapted to: 0.0438329182355754
Iteration:  187
	Trajectories constructed.
	Epsilon: 3.2481486692572528
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2134000000000000
	Step-size adapted to: 0.0434706106705599
Iteration:  188
	Trajectories constructed.
	Epsilon: 3.2328971018737960
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2210000000000000
	Step-size adapted to: 0.0432754324030076
Iteration:  189
	Trajectories constructed.
	Epsilon: 3.2159301643610645
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2204000000000000
	Step-size adapted to: 0.0430682080637579
Iteration:  190
	Trajectories constructed.
	Epsilon: 3.1993292608973771
	Weights computed and normalized.
	Particles

	Trajectories constructed.
	Epsilon: 2.7829774336203137
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2146000000000000
	Step-size adapted to: 0.0338922065607725
Iteration:  224
	Trajectories constructed.
	Epsilon: 2.7717691258659807
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2278000000000000
	Step-size adapted to: 0.0338549456308242
Iteration:  225
	Trajectories constructed.
	Epsilon: 2.7587067495813420
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2278000000000000
	Step-size adapted to: 0.0338177256653643
Iteration:  226
	Trajectories constructed.
	Epsilon: 2.7483100024995295
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.2170000000000000
	Step-size adapted to: 0.0335986233026407
Iteration:  227
	Trajectories constructed.
	Epsilon: 2.7374104078538961
	Weights computed and normalized.
	Particles

	Velocities refreshed.
	Prop Moved: 0.2134000000000000
	Step-size adapted to: 0.0265886675483694
Iteration:  260
	Trajectories constructed.
	Epsilon: 2.4483248707337824
